# Build a simulation using the Python API

Currently, this notbook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.


Import the Simulation class

In [1]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [2]:
my_sim = Simulation()
net = my_sim.network
my_sim.describe_state()

{'uuid': '91c88b2a-caf1-47be-a394-d0c22e5110be',
 'network': {'uuid': 'a9121808-0401-460c-9833-23d4ba91e9bc',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': '25fbe0e9-76e8-4fd7-ad22-da2d2b5a509d', 'accounts': {}}}

## Add nodes

In [3]:
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

In [4]:
my_pc = Computer(hostname="primaite_pc", ip_address="192.168.1.10", subnet_mask="255.255.255.0")
net.add_node(my_pc)
my_server = Server(hostname="google_server", ip_address="192.168.1.11", subnet_mask="255.255.255.0")
net.add_node(my_server)


## Connect the nodes

In [5]:
from primaite.simulator.network.hardware.nodes.host.host_node import NIC
from primaite.simulator.network.hardware.nodes.network.switch import Switch


In [6]:
my_switch = Switch(hostname="switch1", num_ports=12)
net.add_node(my_switch)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)

server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)

net.connect(pc_nic, my_switch.network_interface[1])
net.connect(server_nic, my_switch.network_interface[2])


Link(uuid='2bd19485-0a6b-4878-978b-b082a672d9b9', endpoint_a=NIC(ip_address=IPv4Address('130.1.1.2'), subnet_mask=IPv4Address('255.255.255.0'), uuid='8a628493-83fb-44bf-a1b0-ef19e362ae5f', mac_address='44:89:a5:ce:7f:6f', speed=100, mtu=1500, enabled=False, port_num=2, port_name=None, pcap=None, nmne={}, wake_on_lan=False, gateway='130.1.1.255'), endpoint_b=SwitchPort(uuid='a049bb8f-53d3-4575-b325-dfb55516edcd', mac_address='aa:45:88:e1:13:e5', speed=100, mtu=1500, enabled=False, port_num=2, port_name=None, pcap=None, nmne={}), bandwidth=100.0, current_load=0.0)

## Add files and folders to nodes


In [7]:
from primaite.simulator.file_system.file_type import FileType
from primaite.simulator.file_system.file_system import File
from primaite.simulator.system.core.sys_log import SysLog

In [8]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(File(name="firefox_installer.zip",folder_id="Test", folder_name="downloads" ,file_type=FileType.ZIP, sys_log=SysLog(hostname="Test")))

In [9]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileType.PNG)

File(uuid='3ceeded4-77b9-4a86-949c-73188d5f4c34', name='favicon.ico', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.GOOD: 1>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x000002CCE8EDB790>, deleted=False, folder_id='cbbd3631-a915-400d-bc02-f31f72447ce5', folder_name='root', file_type=<FileType.UNKNOWN: 0>, sim_size=0, real=False, sim_path=None, sim_root=WindowsPath('C:/Projects/PrimAITE/simulation_output/2024-04-09_13-24-30/google_server/fs'), num_access=0, folder=Folder(uuid='cbbd3631-a915-400d-bc02-f31f72447ce5', name='root', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.GOOD: 1>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x000002CCE8EDB790>, deleted=False, files={'3ceeded4-77b9-4a86-949c-73188d5f4c34': File(uuid='3ceeded4-77b9-4a86-949c-73188d5

## Add applications to nodes

In [10]:
from pathlib import Path
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.network.transmission.transport_layer import Port
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.file_system.file_system import FileSystem

# no applications exist yet so we will create our own.
class MSPaint(Application):
    def describe_state(self):
        return super().describe_state()

In [11]:
mspaint = MSPaint(name = "mspaint", health_state_actual=SoftwareHealthState.GOOD, health_state_visible=SoftwareHealthState.GOOD, criticality=SoftwareCriticality.MEDIUM, port=Port.HTTP, protocol = IPProtocol.NONE,operating_state=ApplicationOperatingState.RUNNING,execution_control_status='manual', file_system=FileSystem(sys_log=SysLog(hostname="Test"), sim_root=Path(__name__).parent),)

In [12]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [13]:
from primaite.simulator.domain.account import Account, AccountType


In [14]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [15]:
my_sim.describe_state()

{'uuid': '91c88b2a-caf1-47be-a394-d0c22e5110be',
 'network': {'uuid': 'a9121808-0401-460c-9833-23d4ba91e9bc',
  'nodes': {'primaite_pc': {'uuid': 'dd0e95be-2491-4d5b-8388-df3975a19e8a',
    'hostname': 'primaite_pc',
    'operating_state': 2,
    'NICs': {1: {'uuid': '279e2645-b680-4d2e-b13c-66d5cfacbd38',
      'mac_address': 'bd:76:20:24:cf:04',
      'speed': 100,
      'mtu': 1500,
      'enabled': False,
      'nmne': {},
      'ip_address': '192.168.1.10',
      'subnet_mask': '255.255.255.0',
      'wake_on_lan': False},
     2: {'uuid': '40c0db02-4d14-4826-b49b-e6a521941cec',
      'mac_address': 'd8:b2:0c:af:3f:83',
      'speed': 100,
      'mtu': 1500,
      'enabled': False,
      'nmne': {},
      'ip_address': '130.1.1.1',
      'subnet_mask': '255.255.255.0',
      'wake_on_lan': False}},
    'file_system': {'uuid': '91d3aed7-53c6-471f-b903-9889396be280',
     'folders': {'root': {'uuid': '81bdc04e-9a0d-4306-9a9c-ee926fff6df8',
       'name': 'root',
       'health_statu

In [16]:
import json
json.dumps(my_sim.describe_state())

'{"uuid": "91c88b2a-caf1-47be-a394-d0c22e5110be", "network": {"uuid": "a9121808-0401-460c-9833-23d4ba91e9bc", "nodes": {"primaite_pc": {"uuid": "dd0e95be-2491-4d5b-8388-df3975a19e8a", "hostname": "primaite_pc", "operating_state": 2, "NICs": {"1": {"uuid": "279e2645-b680-4d2e-b13c-66d5cfacbd38", "mac_address": "bd:76:20:24:cf:04", "speed": 100, "mtu": 1500, "enabled": false, "nmne": {}, "ip_address": "192.168.1.10", "subnet_mask": "255.255.255.0", "wake_on_lan": false}, "2": {"uuid": "40c0db02-4d14-4826-b49b-e6a521941cec", "mac_address": "d8:b2:0c:af:3f:83", "speed": 100, "mtu": 1500, "enabled": false, "nmne": {}, "ip_address": "130.1.1.1", "subnet_mask": "255.255.255.0", "wake_on_lan": false}}, "file_system": {"uuid": "91d3aed7-53c6-471f-b903-9889396be280", "folders": {"root": {"uuid": "81bdc04e-9a0d-4306-9a9c-ee926fff6df8", "name": "root", "health_status": 1, "visible_status": 1, "previous_hash": null, "revealed_to_red": false, "files": {}, "deleted_files": {}}, "downloads": {"uuid": 